In [ ]:
%env DSP_CACHEBOOL=true

In [ ]:
%run ./module-00-data-prep.ipynb
train = train
val = val
test = test
len(train), len(val), len(test)

In [ ]:
import logging
import sys

logging.disable(sys.maxsize)

In [ ]:
import dspy

MODEL = "gpt-4o-ai-factory"
lm = dspy.AzureOpenAI(
    api_base="https://aifactorydemo8563613420.openai.azure.com/openai/deployments/gpt-4o-ai-factory/chat/completions?api-version=2023-03-15-preview",
    api_version="2023-03-15-preview",
    deployment_id=MODEL,
)
dspy.configure(lm=lm)

dspy.settings.configure(lm=lm)
dspy.configure(experimental=True)

In [ ]:
from sms_classifier import Input, Output


class SMSClassifierSignature(dspy.Signature):
    """
    Given an SMS text, predict whether it is ham, spam, or smishing.
    Output only the predicted label.
    """

    input: Input = dspy.InputField()
    output: Output = dspy.OutputField()

In [ ]:
class SMSClassifier(dspy.Module):
    def __init__(self, lm):
        self.lm = lm
        super().__init__()
        dspy.configure(lm=lm)
        self.generate_answer = dspy.TypedPredictor(
            SMSClassifierSignature, max_retries=1
        )

    def forward(self, input):
        return self.generate_answer(input=input)


sms_classifier = SMSClassifier(lm)

In [ ]:
from datetime import datetime

from evaluation_helpers import validate_answer
from langfuse_extensions import EvaluateWithLangfuse

session = f"Run-{MODEL}-{int(datetime.now().timestamp())}"
print(session)
# from evaluation_helpers import EvaluateWithLangfuse
evaluator = EvaluateWithLangfuse(
    devset=test, num_threads=1, display_progress=True, run_id=session
)
evaluator(sms_classifier, metric=validate_answer)

In [ ]:
from evaluation_helpers import calculate_metrics, fetch_traces
from sms_classifier import Label

classes = [l.value for l in Label]

In [ ]:
# Run-gpt-4o-ai-factory-1729432239
traces_gpt_4 = fetch_traces(run_id="Run-gpt-4o-ai-factory-1729432239")
metrics_gpt_4 = calculate_metrics(traces_gpt_4, classes)
metrics_gpt_4["macro"]

In [ ]:
train[0].output.label.value